# New Section

In [5]:
import tweepy
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import plotly.graph_objects as go

import pandas as pd

# Secret Credentials
ACCESS_TOKEN = "500435442-ZsyX0WGAgAsiK3mPMwCLVZgXaM4mPeLpOZHh46Uc"
ACCESS_TOKEN_SECRET = "tI6pbzVsXWzNmfUEC07by6ZpJc5d58vu0sKxovxuDQEoY"
CONSUMER_KEY = "zazEfoGYmcJp8IZCQZYeleORe"
CONSUMER_SECRET = "JtTfz6C4WaCNZkzmn02U4GkOoilCyiIVKFFknuLgWD51MeVySm"

auth=OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN,ACCESS_TOKEN_SECRET)
api=tweepy.API(auth)

tweets=[]
retweet=[]
likes=[]

userx='rihanna'

for i in tweepy.Cursor(api.user_timeline, id=userx, tweet_mode='extended').items(200):
    tweets.append(i.full_text)
    retweet.append(i.retweet_count)
    likes.append(i.favorite_count)

Twitter_df2=pd.DataFrame({'tweets':tweets,'likes':likes,'retweet':retweet})


new_docs=list(Twitter_df2['tweets'])

#LOAD MODEL
loaded_vec = CountVectorizer(vocabulary=pickle.load(open("count_vector.pkl", "rb")))
loaded_tfidf = pickle.load(open("tfidf.pkl","rb"))
loaded_model = pickle.load(open("nb_model.pkl","rb"))

news_category=[]

for i in range(len(new_docs)):
    X_new_counts = loaded_vec.transform([new_docs[i]])
    X_new_tfidf = loaded_tfidf.transform(X_new_counts)
    predicted = loaded_model.predict(X_new_tfidf)
    news_category.append(predicted)

x=[]
for i in range(len(news_category)):
    x.append(news_category[i][0])

set_x=list(set(x))
set_x_count=[]

for j in range(len(set_x)):
    set_x_count.append(x.count(set_x[j]))

Tweet_Uni=dict(zip(set_x,set_x_count))

count1=list(Tweet_Uni.values())
size1=[]
for i in count1:
    size1.append(i*2)

colours=[]
c=120
for i in range(len(count1)):
    c=c+2
    colours.append(c)


fig1 = go.Figure(data=[go.Scatter(
    x=set_x,
    y=set_x_count,
    mode='markers',
    marker=dict(
        color=colours,
        size=size1,
        showscale=True
        )
)])
fig1.show()

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning:

Trying to unpickle estimator TfidfTransformer from version 0.24.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning:

Trying to unpickle estimator MultinomialNB from version 0.24.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.



In [14]:
import plotly.io 

plotly.io.write_json(fig1,'twitter.json',validate=True, pretty=False, remove_uids=True)
